In [1]:
import pandas as pd 
import json
import ast
from textblob import TextBlob

In [3]:
# Crea una lista vacía llamada data_list para almacenar los datos del archivo JSON.
data_list = []

# Especifica la ruta del archivo JSON que se va a leer.
file_path = 'australian_user_reviews.json'

# Abre el archivo en modo de lectura ('r') con codificación UTF-8.
with open(file_path, 'r', encoding='utf-8') as file:
    # Itera sobre cada línea en el archivo.
    for line in file:
        try:
            # Intenta evaluar la línea como un diccionario de Python utilizando ast.literal_eval.
            # Agrega el diccionario resultante a la lista data_list.
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            # Si ocurre un error al evaluar la línea, imprime un mensaje de error y continúa con la siguiente línea.
            print(f"Error en la línea: {line}")
            continue

# Crea un DataFrame de pandas a partir de la lista de diccionarios almacenada en data_list.
data_reviews = pd.DataFrame(data_list)

In [ ]:
# Utiliza el método 'explode' para descomponer la columna 'reviews' que contiene listas en varias filas duplicando las demás columnas.
data_reviews1 = data_reviews1.explode(['reviews'])

# Aplica el método 'pd.Series' a la columna 'reviews' para expandir las listas en nuevas columnas.
data_reviews2 = data_reviews1['reviews'].apply(pd.Series)

# Combina los DataFrames 'data_reviews1' y 'data_reviews2' a lo largo de las columnas.
data_reviews3 = pd.concat([data_reviews1, data_reviews2], axis=1)

In [4]:

# Extrae el año de la columna 'posted' y almacénalo en una nueva columna 'year_posted'.
data_reviews3['year_posted'] = data_reviews3['posted'].str.extract('(\d{4})')

# Elimina columnas innecesarias
data_reviews3.drop(['reviews', 'user_url', 'last_edited', 'funny', 'helpful', 'posted'], axis=1, inplace=True)

# Elimina filas con valores nulos en columnas específicas
data_reviews3 = data_reviews3.dropna(subset=['year_posted', 'item_id', 'review', 'recommend'])

# Selecciona columnas específicas y renómbralas.
data_reviews3 = data_reviews3[['user_id', 'item_id', 'review', 'recommend', 'year_posted', 0]]

# Reemplaza valores 'False' y 'True' en la columna 'recommend' con 0 y 1, respectivamente.
data_reviews3['recommend'] = data_reviews3['recommend'].replace({'False': 0, 'True': 1}).astype(int)

# Elimina la última columna, que parece ser una columna residual después de expandir las listas.
del data_reviews3[data_reviews3.columns[-1]]

# Define una función para realizar análisis de sentimientos en la columna 'review'.
def get_sentiment(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity
    if sentiment < -0.1:
        return 0
    elif sentiment > 0.1:
        return 2
    else:
        return 1

# Aplica la función 'get_sentiment' a la columna 'review' y almacena los resultados en una nueva columna 'sentiment_analysis'.
data_reviews3['sentiment_analysis'] = data_reviews3['review'].apply(get_sentiment)

# Elimina la columna original 'review' después de realizar el análisis de sentimientos.
data_reviews3.drop(['review'], axis=1, inplace=True)

# Elimina duplicados en el DataFrame.
data_reviews3 = data_reviews3.drop_duplicates()

# Elimina filas con valores nulos en todo el DataFrame.
data_reviews3 = data_reviews3.dropna()

In [5]:
data_reviews3

,user_id,item_id,recommend,year_posted,sentiment_analysis
0,76561197970982479,1250,1,2011,2
0,76561197970982479,22200,1,2011,2
0,76561197970982479,43110,1,2011,1
1,js41637,251610,1,2014,2
1,js41637,227300,1,2013,1
...,...,...,...,...,...
25764,wayfeng,730,1,2015,1
25765,76561198251004808,253980,1,2015,2
25769,72947282842,730,1,2015,0
25771,ApxLGhost,730,1,2015,2


In [6]:
data_reviews3.to_csv('user_reviews.csv', index=False)

In [2]:
# Crea una lista vacía llamada data_list para almacenar los datos del archivo JSON.
data_list = []

# Especifica la ruta del archivo JSON que se va a leer.
file_path = 'australian_users_items.json'

# Abre el archivo en modo de lectura ('r') con codificación UTF-8.
with open(file_path, 'r', encoding='utf-8') as file:
    # Itera sobre cada línea en el archivo.
    for line in file:
        try:
            # Intenta evaluar la línea como un diccionario de Python utilizando ast.literal_eval.
            # Agrega el diccionario resultante a la lista data_list.
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            # Si ocurre un error al evaluar la línea, imprime un mensaje de error y continúa con la siguiente línea.
            print(f"Error en la línea: {line}")
            continue

# Crea un DataFrame de pandas a partir de la lista de diccionarios almacenada en data_list.
data_items = pd.DataFrame(data_list)


In [3]:

# Utiliza el método 'explode' para descomponer la columna 'items' que contiene listas en varias filas duplicando las demás columnas.
data_items1 = data_items.explode(['items'])

# Utiliza 'pd.json_normalize' para aplanar las columnas JSON anidadas dentro de la columna 'items'.
# Luego, establece el índice de 'data_items2' para que coincida con el índice original de 'data_items1'.
data_items2 = pd.json_normalize(data_items1['items']).set_index(data_items1['items'].index)

# Combina los DataFrames 'data_items1' y 'data_items2' a lo largo de las columnas.
data_items3 = pd.concat([data_items2, data_items1], axis=1)

# Elimina columnas innecesarias: 'items_count', 'user_url', 'item_name', 'steam_id'.
data_items3.drop(['items_count', 'user_url', 'item_name', 'steam_id'], axis=1, inplace=True)

# Elimina filas con valores nulos en columnas específicas: 'item_id' y 'playtime_forever'.
data_items3 = data_items3.dropna(subset=['item_id', 'playtime_forever'])

# Elimina columnas 'items' y 'playtime_2weeks'.
data_items3.drop(['items', 'playtime_2weeks'], axis=1, inplace=True)

# Filtra las filas donde 'playtime_forever' no sea igual a 0.
data_items3 = data_items3[data_items3['playtime_forever'] != 0]

# Selecciona columnas específicas.
data_items3 = data_items3[['user_id', 'item_id', 'playtime_forever']]

# Elimina duplicados en el DataFrame.
data_items3 = data_items3.drop_duplicates()

# Elimina filas con valores nulos en todo el DataFrame.
data_items3 = data_items3.dropna()

In [4]:
data_items3

,user_id,item_id,playtime_forever
0,76561197970982479,10,6.0
0,76561197970982479,30,7.0
0,76561197970982479,300,4733.0
0,76561197970982479,240,1853.0
0,76561197970982479,3830,333.0
...,...,...,...
88308,76561198329548331,304930,677.0
88308,76561198329548331,227940,43.0
88308,76561198329548331,388490,3.0
88308,76561198329548331,521570,4.0


In [ ]:
data_items3.to_csv('user_items.csv', index=False)

In [2]:

# Abre el archivo 'output_steam_games.json' en modo de lectura ('r') con codificación UTF-8.
with open('output_steam_games.json', 'r', encoding='utf-8') as archivo:
    # Carga los datos del archivo utilizando json.loads para cada línea.
    data = [json.loads(line) for line in archivo]

# Crea un DataFrame de pandas a partir de los datos cargados.
data_games = pd.DataFrame(data)

# Elimina las primeras 88,310 filas del DataFrame. 
# Esto puede ser útil si las primeras filas no contienen información relevante o son encabezados no deseados.
data_games.drop(data_games.index[0:88310], inplace=True)

In [3]:
data_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
88314,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [4]:
# Elimina filas con valores nulos en columnas específicas y resetea el indice
data_games.drop(['app_name','early_access','url','reviews_url','tags','publisher','specs'], axis=1, inplace = True)
data_games.reindex()

,genres,title,release_date,price,id,developer
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,Free To Play,643980,Secret Level SRL
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,Free to Play,670290,Poolians.com
88313,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,0.99,767400,彼岸领域
88314,NaN,NaN,NaN,2.99,773570,NaN
...,...,...,...,...,...,...
120440,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,2018-01-04,1.99,773640,"Nikita ""Ghost_RUS"""
120441,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,2018-01-04,4.99,733530,Sacada
120442,"[Indie, Racing, Simulation]",Russian Roads,2018-01-04,1.99,610660,Laush Dmitriy Sergeevich
120443,"[Casual, Indie]",EXIT 2 - Directions,2017-09-02,4.99,658870,"xropi,stev3ns"


In [5]:
# Convierte la columna 'release_date' a formato datetime y extrae el año.
data_games['release_date'] = pd.to_datetime(data_games['release_date'], errors='coerce').dt.year

# Elimina filas con valores nulos en columnas específicas: 'genres', 'title', 'release_date', 'price', 'id', 'developer'.
data_games = data_games.dropna(subset=['genres', 'title', 'release_date', 'price', 'id', 'developer'])

# Convierte la columna 'release_date' a tipo entero y luego a tipo cadena.
# Esto se hace para asegurarse de que 'release_date' sea un valor numérico antes de convertirlo a cadena.
data_games['release_date'] = data_games['release_date'].astype(int)
data_games['release_date'] = data_games['release_date'].astype(str)


In [6]:

# Reemplaza los valores específicos en la columna 'price' con 0.
data_games['price'] = data_games['price'].replace({'Free To Play': 0, 'Free to Play': 0, 'Play for Free!': 0})

# Filtra las filas donde la columna 'price' pueda convertirse a un valor numérico y elimina las demás.
data_games = data_games[pd.to_numeric(data_games['price'], errors='coerce').notnull()]

# Renombra la columna 'id' a 'item_id'.
data_games = data_games.rename(columns={'id': 'item_id'})

In [7]:
# Aplica el método 'pd.Series' a la columna 'genres' para expandir las listas en nuevas columnas.
data_games1 = data_games['genres'].apply(pd.Series)

# Elimina columnas específicas por índice (1, 2, 3, ..., 9).
data_games1.drop([1, 2, 3, 4, 5, 6, 7, 8, 9], axis=1, inplace=True)

In [8]:
# Concatena los DataFrames
data_games2 = pd.concat([data_games, data_games1], axis=1)

In [9]:
# Elimina la columna y renombramos la columna "0" a "genre" 
data_games2.drop(["genres"],axis=1,inplace=True)
data_games2.rename(columns={"0":"genre"},inplace=True)
data_games2


,title,release_date,price,item_id,developer,0
88310,Lost Summoner Kitty,2018,4.99,761140,Kotoshiro,Action
88311,Ironbound,2018,0,643980,Secret Level SRL,Free to Play
88312,Real Pool 3D - Poolians,2017,0,670290,Poolians.com,Casual
88313,弹炸人2222,2017,0.99,767400,彼岸领域,Action
88315,Battle Royale Trainer,2018,3.99,772540,Trickjump Games Ltd,Action
...,...,...,...,...,...,...
120439,Kebab it Up!,2018,1.99,745400,Bidoniera Games,Action
120440,Colony On Mars,2018,1.99,773640,"Nikita ""Ghost_RUS""",Casual
120441,LOGistICAL: South Africa,2018,4.99,733530,Sacada,Casual
120442,Russian Roads,2018,1.99,610660,Laush Dmitriy Sergeevich,Indie


In [10]:
data_games2.to_csv('games.csv', index=False)